# Private Data Guardrails

A banking agent has tools to check exchange rates, check account balances, and send notifications. Exchange rates are public data. Account balances are confidential. When the agent accesses confidential data, external connectivity is automatically blocked for the rest of the session.

This notebook uses a multi-turn conversation to show that the same CONNECT tool works before sensitive data is loaded, and stops working after.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.agents.utils import nodes_to_message_history
from agentic_patterns.core.compliance.private_data import DataSensitivity, PrivateData
from agentic_patterns.core.tools import ToolPermission, tool_permission

## Tools

Three tools with different data sensitivity profiles. `get_exchange_rates` returns public market data and does not tag the session. `get_balance` returns confidential account data and tags the session as containing private data. `send_notification` reaches an external email service.

In [ ]:
@tool_permission(ToolPermission.READ)
def get_exchange_rates(base_currency: str) -> dict:
    """Get current exchange rates for a base currency."""
    print(f"Fetching exchange rates for {base_currency}")
    rates = {
        "EUR": {"USD": 1.08, "GBP": 0.86, "JPY": 162.5},
        "USD": {"EUR": 0.93, "GBP": 0.79, "JPY": 150.2},
    }
    return rates.get(base_currency, {"error": f"Unknown currency: {base_currency}"})


@tool_permission(ToolPermission.READ)
def get_balance(account_id: str) -> dict:
    """Get account balance."""
    # In a real system, this tool would query a banking API or database.
    # The returned data is not written to the workspace, but it enters
    # the LLM's context window as a tool result -- and from there the
    # agent can pass it to any other tool, including CONNECT tools that
    # send data externally. Tagging the session here prevents that.
    print(f"Reading balance for account: {account_id}")
    pd = PrivateData()
    pd.add_private_dataset(f"account:{account_id}", DataSensitivity.CONFIDENTIAL)
    return {"account_id": account_id, "balance": 15420.50, "currency": "EUR"}


@tool_permission(ToolPermission.WRITE, ToolPermission.CONNECT)
def send_notification(email: str, subject: str, body: str) -> str:
    """Send a notification email to an external address."""
    print(f"Sending email to {email}: {subject}")
    return f"Email sent to {email}: {subject}"


tools = [get_exchange_rates, get_balance, send_notification]

In [ ]:
agent = get_agent(tools=tools)

## Turn 1: Public data -- notification works

The agent checks exchange rates (public) and sends a notification. No sensitive data has been loaded, so the CONNECT tool works.

In [ ]:
prompt_1 = "Get the EUR exchange rates and email a summary to trader@bank.com with subject 'Daily EUR rates'"

run_1, nodes_1 = await run_agent(agent, prompt_1, verbose=True)
print(f"\nResult: {run_1.result.output}")

In [ ]:
# Confirm: no private data in the session yet
pd = PrivateData()
print(f"Has private data: {pd.has_private_data}")

## Turn 2: Confidential data enters the session

The agent checks an account balance. `get_balance` returns confidential customer data and tags the session as private. Then the agent tries to send a notification -- the `@tool_permission` decorator blocks the CONNECT tool.

In [ ]:
from agentic_patterns.core.tools.permissions import ToolPermissionError

message_history = nodes_to_message_history(nodes_1)

prompt_2 = "Now check the balance for account ACC-7291 and email it to client@example.com with subject 'Your balance'"

try:
    run_2, nodes_2 = await run_agent(
        agent, prompt_2, message_history=message_history, verbose=True
    )
    print(f"\nResult: {run_2.result.output}")
except ToolPermissionError as e:
    print(f"\nGuardrail activated: {e}")
    # Capture partial nodes so Turn 3 can continue the conversation
    nodes_2 = nodes_1

In [ ]:
# The session is now tagged as private
pd = PrivateData()
print(f"Has private data: {pd.has_private_data}")
print(f"Sensitivity: {pd.sensitivity}")
print(f"Datasets: {pd.get_private_datasets()}")

## Turn 3: The guardrail persists

Even a request involving only public data is now blocked from external connectivity. The session has been contaminated by confidential data from Turn 2, and the ratchet never goes back.

In [ ]:
message_history = nodes_to_message_history(nodes_2)

prompt_3 = "Get the USD exchange rates and email them to trader@bank.com with subject 'USD update'"

try:
    run_3, nodes_3 = await run_agent(
        agent, prompt_3, message_history=message_history, verbose=True
    )
    print(f"\nResult: {run_3.result.output}")
except ToolPermissionError as e:
    print(f"\nGuardrail activated: {e}")

Turn 3 demonstrates the ratchet principle: once private data enters the session, external connectivity is permanently blocked. The same request that succeeded in Turn 1 now fails -- not because the exchange rates are sensitive, but because the session's context may still contain confidential data from Turn 2.

## Cleanup

In [ ]:
pd = PrivateData()
pd.has_private_data = False